### HyDE (Hypothetical Document Embeddings)

#### 🧠 ¿Qué es HyDE?

**HyDE (Hypothetical Document Embeddings)** es una técnica de recuperación donde, en lugar de hacer embedding de la consulta del usuario directamente, primero generas una **respuesta hipotética** (documento) a la consulta usando un LLM — y luego haces embedding de ese documento hipotético para buscar en tu vector store.

#### 🎯 ¿Cómo funciona HyDE?

**Flujo tradicional (sin HyDE):**
```
Consulta Usuario → Embedding → Búsqueda Vector Store → Documentos Relevantes
```

**Flujo con HyDE:**
```
Consulta Usuario → LLM genera respuesta hipotética → Embedding de la respuesta → Búsqueda Vector Store → Documentos Relevantes
```

#### ✅ ¿Cuándo usar HyDE?

HyDE cierra la brecha entre la intención del usuario y el contenido relevante, especialmente cuando:

1. **Las consultas son cortas** - "Steve Jobs despedido"
2. **Hay desajuste de lenguaje** entre consulta y documentos (pregunta vs respuesta)
3. **Quieres recuperar basado en contenido de respuesta**, no en palabras de pregunta
4. **Vocabulario diferente** - Usuario pregunta con términos simples, documentos usan terminología técnica

#### 🔍 Ejemplo:

**Consulta:** "¿Cuándo despidieron a Steve Jobs de Apple?"

**Sin HyDE:** Se busca directamente el embedding de la pregunta
- Puede no encontrar documentos que contengan la respuesta pero no la pregunta exacta

**Con HyDE:** 
1. LLM genera respuesta hipotética: "Steve Jobs fue despedido de Apple en septiembre de 1985 debido a conflictos con la junta directiva..."
2. Se hace embedding de esta respuesta hipotética
3. Se buscan documentos similares a esta respuesta
4. Mayor probabilidad de encontrar documentos relevantes que contengan información similar

#### 💡 Ventajas:
- ✅ Mejora recuperación cuando hay desajuste semántico
- ✅ Funciona bien con consultas cortas
- ✅ Captura el formato y estilo de las respuestas esperadas
- ✅ Reduce el problema de "vocabulary mismatch"

#### ⚠️ Consideraciones:
- Requiere llamada adicional al LLM (más latencia y costo)
- La calidad depende de la respuesta hipotética generada
- Puede no ser necesario si los documentos y consultas ya están bien alineados

In [ ]:
# Importación de librerías necesarias para implementar HyDE

# WikipediaLoader: Para cargar artículos de Wikipedia automáticamente
from langchain_community.document_loaders import WikipediaLoader

# RecursiveCharacterTextSplitter: Para dividir texto en fragmentos (chunks) manejables
from langchain.text_splitter import RecursiveCharacterTextSplitter

# HuggingFaceEmbeddings: Para generar embeddings vectoriales usando modelos de HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# Chroma: Base de datos vectorial para almacenar y buscar embeddings
from langchain.vectorstores import Chroma

In [ ]:
# Paso 1: Cargar y fragmentar el dataset desde Wikipedia

# Configurar parámetros de fragmentación
# chunk_size=300: Cada fragmento tendrá máximo 300 caracteres
chunk_size = 300

# chunk_overlap=100: Habrá una superposición de 100 caracteres entre fragmentos
# Una superposición mayor (100 vs 50) ayuda a mantener más contexto
chunk_overlap = 100

# Cargar datos de Wikipedia sobre Steve Jobs
# query="Steve Jobs": Busca artículos relacionados con Steve Jobs
# load_max_docs=5: Carga máximo 5 documentos de Wikipedia
loader = WikipediaLoader(query="Steve Jobs", load_max_docs=5)

# load() descarga y devuelve los artículos de Wikipedia
documents = loader.load()

# Crear un splitter para dividir el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap
)

# Dividir los documentos de Wikipedia en fragmentos más pequeños
docs = text_splitter.split_documents(documents=documents)

# Mostrar los fragmentos generados
docs

In [ ]:
# Paso 2: Construir el vector store con embeddings estándar

# Importar FAISS para búsqueda de similitud eficiente
from langchain.vectorstores import FAISS

# Inicializar el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo compacto y eficiente (384 dimensiones)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Crear el vector store FAISS a partir de los documentos fragmentados
# FAISS convierte cada fragmento en un vector para búsqueda rápida
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
# Paso 3: Configurar el LLM que generará las respuestas hipotéticas

# Importar módulos para manejo de variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Importar función para inicializar modelos de chat
from langchain.chat_models import init_chat_model

# Establecer la API key de Groq desde las variables de entorno
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Inicializar el LLM usando Gemma2-9B-IT de Groq
# Este modelo generará las respuestas hipotéticas para HyDE
llm = init_chat_model("groq:gemma2-9b-it")

In [ ]:
# Paso 4: Crear vector store persistente con ChromaDB

# Importar Chroma para almacenamiento vectorial persistente
from langchain.vectorstores import Chroma

# Crear vector store ChromaDB a partir de los documentos
# documents=docs: Fragmentos de Wikipedia sobre Steve Jobs
# embedding=embeddings: Modelo de embeddings HuggingFace
# persist_directory: Directorio donde se guardarán los embeddings de forma persistente
#                    Esto permite reutilizar los embeddings sin recalcularlos
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="output/steve_jobs_for_hyde.db"
)

# Crear el recuperador base (sin HyDE aún)
# search_kwargs={"k": 5}: Recuperar los 5 documentos más similares
base_retriever = db.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Paso 5: Implementar función para generar documentos hipotéticos (HyDE manual)

# Importar parser para convertir salida del LLM a string
from langchain_core.output_parsers import StrOutputParser

# Importar templates para crear prompts estructurados
from langchain.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate

def get_hyde_doc(query):
    """
    Genera un documento hipotético (respuesta imaginaria) para una consulta.
    
    Este es el núcleo de la técnica HyDE:
    1. Toma la consulta del usuario
    2. Pide al LLM que imagine una respuesta detallada
    3. Devuelve esa respuesta hipotética
    
    Args:
        query (str): La consulta del usuario
    
    Returns:
        str: Documento hipotético generado por el LLM
    """
    
    # Plantilla que instruye al LLM a generar una respuesta hipotética
    # El LLM debe imaginar que es un experto escribiendo una explicación detallada
    template = """Imagina que eres un experto escribiendo una explicación detallada sobre el tema: '{query}'
    Crea una respuesta hipotética para el tema"""

    # Crear mensaje del sistema con la plantilla
    system_message_prompt = SystemMessagePromptTemplate.from_template(template=template)
    
    # Crear prompt de chat completo
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
    
    # Formatear el prompt con la consulta específica
    messages = chat_prompt.format_prompt(query=query).to_messages()
    
    # Mostrar los mensajes que se enviarán al LLM (para debug)
    print(messages)
    
    # Invocar el LLM para generar la respuesta hipotética
    response = llm.invoke(messages)
    
    # Extraer el contenido de texto de la respuesta
    hypo_doc = response.content
    
    # Devolver el documento hipotético generado
    return hypo_doc

In [ ]:
# Paso 6: Probar la generación de documento hipotético

# Definir una consulta sobre cuándo despidieron a Steve Jobs
query = '¿Cuándo despidieron a Steve Jobs de Apple?'

# Generar y mostrar el documento hipotético
# Observa cómo el LLM crea una respuesta imaginaria detallada
# Esta respuesta hipotética se usará para buscar documentos similares
print("📄 Documento Hipotético Generado:")
print("="*80)
print(get_hyde_doc(query=query))

In [ ]:
# Paso 7: Usar HyDE para recuperar documentos relevantes

# Flujo completo de HyDE:
# 1. get_hyde_doc(query) genera una respuesta hipotética
# 2. base_retriever.invoke() busca documentos similares a esa respuesta hipotética
# 3. Los documentos recuperados son probablemente más relevantes que si buscáramos la consulta directamente

matched_doc = base_retriever.invoke(get_hyde_doc(query))

# Mostrar los documentos recuperados usando HyDE
print("🔍 Documentos Recuperados con HyDE:")
print("="*80)
print(matched_doc)

### 🔧 Implementación con LangChain: HypotheticalDocumentEmbedder

LangChain proporciona una clase integrada llamada `HypotheticalDocumentEmbedder` que implementa HyDE de forma más elegante y automática.

**Ventajas del HypotheticalDocumentEmbedder:**
- ✅ Integración directa en el pipeline de embeddings
- ✅ Prompts pre-configurados para diferentes tipos de búsqueda
- ✅ Más fácil de usar y mantener
- ✅ Automáticamente genera documento hipotético antes de hacer embedding

In [ ]:
# Paso 8: Implementar HyDE usando la clase de LangChain

# Importar el HypotheticalDocumentEmbedder de LangChain
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder

# Importar librerías necesarias
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

# Cargar el dataset sobre LangChain y CrewAI
loader = TextLoader("langchain_crewai_dataset.txt")

# load() devuelve una lista de documentos
docs = loader.load()

# Crear splitter para dividir en fragmentos
# chunk_size=300: Fragmentos de máximo 300 caracteres
# chunk_overlap=50: Superposición de 50 caracteres entre fragmentos
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Dividir los documentos en chunks
chunks = splitter.split_documents(docs)

In [ ]:
# Paso 9: Configurar embeddings base

# Inicializar embeddings HuggingFace que se usarán como base
# Estos embeddings se aplicarán al documento hipotético generado por HyDE
base_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### 📋 Tipos de Prompts Disponibles en HyDE

Según la documentación oficial de LangChain y el código fuente (PROMPT_MAP), las opciones de `prompt_key` predeterminadas son:

1. **`web_search`** - Búsqueda web general (recomendado para la mayoría de casos)
2. **`sci_fact`** - Hechos científicos
3. **`arguana`** - Argumentación y debate
4. **`trec_covid`** - Información médica/COVID
5. **`fiqa`** - Finanzas y preguntas financieras
6. **`dbpedia_entity`** - Entidades de DBpedia
7. **`trec_news`** - Noticias y artículos periodísticos
8. **`mr_tydi`** - Búsqueda multilingüe

**¿Cómo elegir el prompt_key correcto?**
- **web_search**: Uso general, preguntas variadas (DEFAULT)
- **sci_fact**: Preguntas científicas o técnicas
- **fiqa**: Preguntas sobre finanzas, inversiones, economía
- **trec_news**: Búsqueda en noticias o artículos

Cada `prompt_key` tiene un prompt optimizado para generar documentos hipotéticos en ese dominio específico.

In [ ]:
# Paso 10: Crear HyDE Embedder usando prompt_key predeterminado

# HypotheticalDocumentEmbedder.from_llm() crea un embedder que:
# 1. Toma una consulta
# 2. Usa el LLM para generar un documento hipotético
# 3. Genera embeddings del documento hipotético (no de la consulta original)

hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,  # LLM que generará los documentos hipotéticos
    base_embeddings=base_embeddings,  # Embeddings que se aplicarán al documento hipotético
    prompt_key="web_search"  # Tipo de prompt optimizado para búsqueda web general
)

In [ ]:
# Paso 11: Crear vector store con HyDE embeddings

# Crear ChromaDB usando el HypotheticalDocumentEmbedder
# IMPORTANTE: Ahora los embeddings se generan usando HyDE:
# - Durante la indexación: Se almacenan embeddings normales de los documentos
# - Durante la búsqueda: La consulta se convierte en documento hipotético antes de hacer embedding

vectorstore = Chroma.from_documents(
    documents=chunks,  # Fragmentos del dataset
    embedding=hyde_embedding_function,  # Función de embedding con HyDE integrado
    persist_directory="output/langchain"  # Directorio de persistencia
)

In [ ]:
# Paso 12: Crear prompt para generación de respuestas RAG

# Esta plantilla define cómo el LLM usará el contexto recuperado para responder
rag_prompt = PromptTemplate.from_template("""
Usa el contexto a continuación para responder la pregunta.

Contexto:
{context}

Pregunta: {input}
""")

# Crear la cadena de documentos que combina LLM con el prompt
# Esta cadena toma documentos recuperados y genera una respuesta
rag_chain = create_stuff_documents_chain(llm=llm, prompt=rag_prompt)

In [ ]:
# Paso 13: Construir el pipeline RAG completo con HyDE

def hyde_rag_pipeline(query):
    """
    Pipeline RAG completo que usa HyDE para mejorar la recuperación.
    
    Flujo del pipeline:
    1. La consulta se convierte automáticamente en documento hipotético (por HyDE)
    2. Se hace embedding del documento hipotético
    3. Se buscan documentos similares al documento hipotético
    4. Se genera respuesta usando los documentos recuperados
    
    Args:
        query (str): La consulta del usuario
    
    Returns:
        str: Respuesta generada por el LLM basada en documentos recuperados con HyDE
    """
    
    # 1. Buscar documentos similares usando HyDE
    # similarity_search() automáticamente:
    #    a. Genera documento hipotético de la consulta (usando el LLM)
    #    b. Hace embedding del documento hipotético
    #    c. Busca los k documentos más similares
    matched_docs = vectorstore.similarity_search(query, k=4)
    
    # Mostrar los documentos recuperados (para debug)
    print("🔍 Documentos recuperados con HyDE:")
    print(matched_docs)
    print("\n" + "="*80 + "\n")
    
    # 2. Generar respuesta usando los documentos recuperados como contexto
    response = rag_chain.invoke({
        "input": query,  # Consulta original del usuario
        "context": matched_docs  # Documentos recuperados con HyDE
    })
    
    # 3. Devolver la respuesta generada
    return response

In [ ]:
# Paso 14: Ejecutar consulta de ejemplo con HyDE

# Definir una consulta sobre módulos de memoria en LangChain
query = "¿Qué módulos de memoria proporciona LangChain?"

# Invocar el pipeline RAG con HyDE
# HyDE automáticamente:
# 1. Generará una respuesta hipotética a esta pregunta
# 2. Buscará documentos similares a esa respuesta hipotética
# 3. Usará esos documentos para generar la respuesta final
answer = hyde_rag_pipeline(query)

# Mostrar la respuesta final
print("✅ Respuesta Final:")
print(answer)

### 🎨 Prompt Personalizado para HyDE

Además de usar los `prompt_key` predeterminados, puedes crear tu propio prompt personalizado usando el parámetro `custom_prompt`.

**¿Cuándo usar un prompt personalizado?**
- Cuando ninguno de los prompt_key predeterminados se ajusta a tu dominio
- Cuando quieres control total sobre cómo se genera el documento hipotético
- Para experimentar con diferentes estilos de respuestas hipotéticas

In [ ]:
# Paso 15: Crear HyDE Embedder con prompt personalizado

# Importar PromptTemplate para crear plantillas personalizadas
from langchain.prompts import PromptTemplate

# Definir un prompt personalizado para generar documentos hipotéticos
# Este prompt es más conciso que los predeterminados
custom = PromptTemplate.from_template(
    "Genera una respuesta hipotética concisa para este tema: {query}"
)

# Crear HyDE Embedder usando el prompt personalizado
hyde_embedding_function = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,  # LLM para generar documentos hipotéticos
    base_embeddings=base_embeddings,  # Embeddings base
    custom_prompt=custom  # Usar prompt personalizado en lugar de prompt_key
)

# NOTA: Puedes usar este hyde_embedding_function de la misma forma que antes
# La única diferencia es que usará tu prompt personalizado para generar
# documentos hipotéticos más concisos

### 📊 Resumen: HyDE (Hypothetical Document Embeddings)

#### 🔄 Comparación: Búsqueda Tradicional vs HyDE

**Búsqueda Tradicional:**
```
Consulta: "¿Cuándo despidieron a Steve Jobs?"
    ↓ (embedding directo)
Vector de la pregunta
    ↓ (búsqueda)
Documentos que contienen preguntas similares o palabras clave
```

**Búsqueda con HyDE:**
```
Consulta: "¿Cuándo despidieron a Steve Jobs?"
    ↓ (LLM genera respuesta hipotética)
"Steve Jobs fue despedido de Apple en septiembre de 1985..."
    ↓ (embedding de la respuesta)
Vector de la respuesta hipotética
    ↓ (búsqueda)
Documentos que contienen respuestas similares
```

#### ✅ Ventajas de HyDE

1. **Resuelve el problema de vocabulary mismatch**
   - Usuario: "¿Cuándo echaron a Steve Jobs?"
   - Documento: "Jobs fue despedido en 1985"
   - HyDE genera: "despedido en 1985" → mejor match

2. **Mejora recuperación con consultas cortas**
   - Consulta corta: "memoria LangChain"
   - HyDE expande: "LangChain ofrece ConversationBufferMemory y ConversationSummaryMemory..."
   - Más contexto = mejor recuperación

3. **Alinea formato pregunta-respuesta**
   - Busca respuestas similares a una respuesta hipotética
   - Más natural que buscar documentos similares a una pregunta

#### ⚠️ Desventajas de HyDE

1. **Latencia aumentada**
   - Requiere llamada adicional al LLM antes de la búsqueda
   - ~500ms-2s de overhead dependiendo del LLM

2. **Costo adicional**
   - Cada búsqueda consume tokens del LLM
   - Para APIs de pago (OpenAI, etc.), aumenta el costo

3. **Dependencia de la calidad del LLM**
   - Si el documento hipotético es incorrecto, la búsqueda será peor
   - Requiere un LLM razonablemente bueno

4. **No siempre necesario**
   - Si documentos y consultas ya están alineados, HyDE puede no ayudar
   - Puede ser overkill para bases de conocimiento bien estructuradas

#### 🎯 Casos de Uso Ideales

**✅ Cuándo USAR HyDE:**
- Consultas en lenguaje natural vs documentos técnicos
- Preguntas cortas que necesitan expansión semántica
- Bases de conocimiento con vocabulario diferente al usuario
- FAQs donde buscas respuestas, no preguntas
- Dominios especializados (medicina, leyes, finanzas)

**❌ Cuándo NO usar HyDE:**
- Búsqueda de palabras clave exactas
- Aplicaciones de latencia crítica
- Presupuesto muy limitado
- Documentos y consultas ya bien alineados
- Búsqueda de código (donde sintaxis exacta importa)

#### 🔧 Opciones de Implementación

**1. Manual (más control):**
```python
hyde_doc = llm.invoke("Genera respuesta para: {query}")
results = vectorstore.similarity_search(hyde_doc)
```

**2. LangChain HypotheticalDocumentEmbedder (recomendado):**
```python
hyde_embedder = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=embeddings,
    prompt_key="web_search"  # o custom_prompt=...
)
```

#### 📈 Comparación con Otras Técnicas

| Técnica | Latencia | Costo | Precisión | Uso Ideal |
|---------|----------|-------|-----------|----------|
| **Búsqueda Simple** | Baja | Bajo | ⭐⭐⭐ | Documentos y consultas alineados |
| **Query Expansion** | Media | Medio | ⭐⭐⭐⭐ | Agregar sinónimos |
| **HyDE** | Alta | Alto | ⭐⭐⭐⭐⭐ | Vocabulary mismatch |
| **Query Decomposition** | Alta | Alto | ⭐⭐⭐⭐⭐ | Consultas complejas |

#### 💡 Consejos Prácticos

1. **Experimenta con diferentes prompt_keys** - Cada dominio puede funcionar mejor con diferentes prompts
2. **Combina con otras técnicas** - HyDE + Reranking puede dar excelentes resultados
3. **Cachea documentos hipotéticos** - Si las consultas se repiten, guarda el documento hipotético
4. **Monitorea la calidad** - Revisa qué documentos hipotéticos se generan
5. **A/B testing** - Compara resultados con y sin HyDE en tu caso de uso específico